In [1]:
# Load packages
import os
import pandas as pd
import numpy as np
 # Required for basic python plotting functionality
import matplotlib.pyplot as plt
# Required for formatting dates later in the case
import datetime
import matplotlib.dates as mdates
# Advanced plotting functionality with seaborn
import seaborn as sns

sns.set(style="whitegrid")  # can set style depending on how you'd like it to look
# Load the data into python
energy_consumption_file = os.path.join(os.getcwd(), "./districts_info.csv")
# electricity_generation_file = os.path.join(
#     os.getcwd(), "data/electricity_generation.csv"
# )
# print(energy_consumption_file)
energy_df = pd.read_csv(energy_consumption_file)

temp1 = os.path.join(os.getcwd(), "./engagement_data/1000.csv")
# electricity_generation_file = os.path.join(
#     os.getcwd(), "data/electricity_generation.csv"
# )
# print(energy_consumption_file)
otherDF = pd.read_csv(temp1)
# electricity_df = pd.read_csv(electricity_generation_file)

In [2]:
# Get data
discritDF = os.path.join(os.getcwd(), "./districts_info.csv")
discritDF = pd.read_csv(discritDF)

engagmentDF = os.path.join(os.getcwd(), "./engagement_data/1000.csv")
engagmentDF = pd.read_csv(engagmentDF)

productInfoDF = os.path.join(os.getcwd(), "./products_info.csv")

productInfoDF = pd.read_csv(productInfoDF)

<h1>Begining of Preprocessing Data</h1>

In [17]:
def preProcessingData():
    policyDataFrame = os.path.join(os.getcwd(), "./COVID-19 US state policy 3_29_2021.csv")
    policyDataFrame = pd.read_csv(policyDataFrame)
    originalStateColumn = policyDataFrame["STATE"]
    policyDataFrame = policyDataFrame.T
    dfs = [x for _, x in policyDataFrame.groupby(1)] # this allowed me to make 26 different series so i can take a look at each category carefully 
    listofDfs = []
    for miniDf in dfs: # This put the category description back 
        tempdf = miniDf.T
        tempdf["STATE"] = originalStateColumn
        listofDfs.append(tempdf)
    return listofDfs
    for k in range (0, len(listofDfs)): # This allowed me to parse through each category dataframe to inspect it without having to save it onto my hard-drive giving me O(1) Space :) 
        temp = pd.DataFrame(listofDfs[k].iloc[0])
        print(listofDfs[k].T.reset_index()) 
def getMeDataFrame(k :int  , listofDfs : pd.DataFrame):
    temp = pd.DataFrame(listofDfs[k].iloc[0])
    return listofDfs[k].T.reset_index()
    print(listofDfs[k].T.reset_index()) 


In [23]:
listofDfs = preProcessingData()
arbitraryNum = 22
getMeDataFrame(arbitraryNum,listofDfs) # This allowed me to parse through each category dataframe to inspect it without having to save it onto my hard-drive giving me O(1) Space :) 


,index,0,1,2,3,4,5,6,7,8,...,47,48,49,50,51,52,53,54,55,56
0,CLBAR3,Close Bars (x3),third_closures,start,date,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
1,CLRST3,Close Indoor Dining (x3),third_closures,start,date,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,END_CLRST3,Reopen Indoor Dining (x3),third_closures,end,date,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,STATE,State,category,type,unit,Alabama,Alaska,Arizona,Arkansas,California,...,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,NaN,NaN


This Preprossing of data allowed us to get these following categories of interest
- Reopening,food_security, masks,physical_distance_closure,physical_distance_closures,pre_covid_policy
- population_density,quarantines,reopening,second_closures,shelter,state_characteristics, State of emergency issued	

Along with These topics within each category : 
- Category [minimum_wage] = MINWAGEJAN2020	MINWAGEJUL2020	MINWAGESEP2020	MINWAGEOCT2020	TIPMINWAGE2020 (currently not focusing on this)
- Category [unemployment] = UIMAXDUR (currently not focusing on this)

<h1>End of pre-processing</h1>

In [61]:
policyDataFrame = os.path.join(os.getcwd(), "./COVID-19 US state policy 3_29_2021.csv")
policyDataFrame = pd.read_csv(policyDataFrame)
categoriesOfInterest = ["category","Reopening","food_security", "masks","physical_distance_closure","physical_distance_closures","pre_covid_policy"
"population_density","quarantines","reopening","second_closures","shelter,state_characteristics", "State of emergency issued"]

In [62]:
policyDataFrame= policyDataFrame.T[policyDataFrame.T[1].isin( categoriesOfInterest)].T
policyDataFrame.drop([55, 56], inplace=True)

In [ ]:
temp = pd.DataFrame(policyDataFrame.columns.tolist())
temp["Yes/No"] = np.nan
# print(policyDataFrame.columns.tolist())

In [ ]:
alice_columns = df.loc[:, df.iloc[0] == 'Alice']


In [ ]:
# DifferentCategories = 
pd.DataFrame(policyDataFrame.iloc[1]).groupby(by=1).value_counts()



In [ ]:
pd.DataFrame(DifferentCategories[1].unique())

In [ ]:
Categories_of_interest = [0,1,2,]

In [ ]:
policyDataFrame.loc[:, policyDataFrame.iloc[1] == 'state_of_emergency']



In [ ]:
discritDF.shape
discritDF.head()

In [ ]:
engagmentDF.head()

In [ ]:
productInfoDF.head()

In [ ]:
(discritDF.shape)
print(engagmentDF.shape)
print(productInfoDF.shape)


In [ ]:
stateDisc = discritDF[["state","district_id"]].groupby(["state"])["district_id"].count().to_frame()

In [ ]:
productInfoDF["Sector(s)"].unique()

In [ ]:
temp1 = os.path.join(os.getcwd(), "./engagement_data/1000.csv")
engData1000 = pd.read_csv(temp1)

temp2 = os.path.join(os.getcwd(), "./engagement_data/1039.csv")
engData1039 = pd.read_csv(temp2)

temp3 = os.path.join(os.getcwd(), "./engagement_data/1044.csv") 
engData1044 = pd.read_csv(temp3)

In [ ]:
len(engData1000["time"].unique())

In [ ]:
productInfoDF["Primary Essential Function"].unique()
# LC = Learning & Curriculum,
# CM = Classroom Management, 
# and SDO = School & District Operations. 
# Each of these categories have multiple sub-categories 
# with which the products were labeled |

In [ ]:
temp4 = os.path.join(os.getcwd(), "./engagement_data/8815.csv") 
engData8815 = pd.read_csv(temp4)
len(engData8815["time"].unique())

In [ ]:
DennisDF = pd.read_csv(os.path.join(os.getcwd(), "./ACSST1Y2021.S1501-2023-03-08T022240.csv"))

In [ ]:
print(DennisDF.columns)

In [ ]:
# discritDF[["locale","pct_free/reduced","state"]].groupby(["locale","pct_free/reduced"]).count()
discritDF.groupby(["state","pct_free/reduced"]).count()

Engagement_data 
<p1>The engagement data are aggregated at school district level, and each file in the folder `engagement_data` represents data from one school district. The 4-digit file name represents `district_id` which can be used to link to district information in `district_info.csv`. The `lp_id` can be used to link to product information in `product_info.csv`.
</p1>